In [2]:
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()  # automatically looks for .env in the current directory
google_api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool



In [7]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

### MCP 

Model Context Protocol (MCP) is an open standard that lets agents use community-built integrations. Instead of writing your own integrations and API clients, just connect to an existing MCP server.


MCP enables agents to:

✅ Access live, external data from databases, APIs, and services without custom integration code

✅ Leverage community-built tools with standardized interfaces

✅ Scale capabilities by connecting to multiple specialized servers


Using MCP with Your Agent¶

The workflow is simple:

* Choose an MCP Server and tool
* Create the MCP Toolset (configure connection)
* Add it to your agent
* Run and test the agent




#### Step 1

For this demo, we'll use the Everything MCP Server - an npm package (@modelcontextprotocol/server-everything) designed for testing MCP integrations.

It provides a getTinyImage tool that returns a simple test image (16x16 pixels, Base64-encoded). Find more servers: modelcontextprotocol.io/examples

#### Step 2

Step 2: Create the MCP Toolset

The McpToolset is used to integrate an ADK Agent with an MCP Server.

What the code does:

* Uses npx (Node package runner) to run the MCP server
* Connects to @modelcontextprotocol/server-everything
* Filters to only use the getTinyImage tool (the server has others, but we only nee

In [8]:
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",  # Run MCP server via npx
            args=[
                "-y",  # Argument for npx to auto-confirm install
                "@modelcontextprotocol/server-everything",
            ],
            tool_filter=["getTinyImage"],
        ),
        timeout=30,
    )
)


#### Behind the scenes:

1. Server Launch: ADK runs npx -y @modelcontextprotocol/server-everything
2. Handshake: Establishes stdio communication channel
3. Tool Discovery: Server tells ADK: "I provide getTinyImage" functionality
4. Integration: Tools appear in agent's tool list automatically
5. Execution: When agent calls getTinyImage(), ADK forwards to MCP server
6. Response: Server result is returned to agent seamlessly
7. Why This Matters: You get instant access to tools without writing integration code

In [9]:
image_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="image_agent",
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_image_server],
)

In [10]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [ ]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [ ]:
response = await runner.run_debug("Provide a sample tiny image", verbose=True)

In [ ]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

#### Other MCP Servers

Kaggle MCP Server - For dataset and notebook operations¶
Kaggle provides an MCP server that lets your agents interact with Kaggle datasets, notebooks, and competitions.

In [ ]:
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)

### Section 3: Long-Running Operations (Human-in-the-Loop)

When to use Long-Running Operations:

💰 Financial transactions requiring approval (transfers, purchases)

🗑️ Bulk operations (delete 1000 records - confirm first!)

📋 Compliance checkpoints (regulatory approval needed)

💸 High-cost actions (spin up 50 servers - are you sure?)

⚠️ Irreversible operations (permanently delete account)


### Example:

Let's build a shipping coordinator agent with one tool that:

Auto-approves small orders (≤5 containers)
Pauses and asks for approval on large orders (>5 containers)
Completes or cancels based on the approval decision
This demonstrates the core long-running operation pattern: pause → wait for human input → resume.



#### The Shipping Tool with Approval Logic¶
Here's the complete function.

The ToolContext Parameter
Notice the function signature includes tool_context: ToolContext. ADK automatically provides this object when your tool runs. It gives you two key capabilities:

Request approval: Call tool_context.request_confirmation()
Check approval status: Read tool_context.tool_confirmation

In [ ]:
LARGE_ORDER_THRESHOLD = 5


def place_shipping_order(
    num_containers: int, destination: str, tool_context: ToolContext
) -> dict:
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_containers": num_containers, "destination": destination},
        )
        return {  # This is sent to the Agent
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }

In [ ]:
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)],
)

print("✅ Shipping Agent created!")

### Step 2: Wrap in resumable App

The problem: A regular LlmAgent is stateless - each call is independent with no memory of previous interactions. If a tool requests approval, the agent can't remember what it was doing.

The solution: Wrap your agent in an App with resumability enabled. The App adds a persistence layer that saves and restores state.

What gets saved when a tool pauses:

All conversation messages so far
Which tool was called (place_shipping_order)
Tool parameters (10 containers, Rotterdam)
Where exactly it paused (waiting for approval)
When you resume, the App loads this saved state so the agent continues exactly where it left off - as if no time passed.

In [ ]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

print("✅ Resumable app created!")

In [ ]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service,
)

print("✅ Runner created!")

#### Recap: Your Pausable shipping agent is compete!

1. A tool that can pause for approval. The tool return the status to the agent.
2. An Agent that uses this tool
3. A resumable app that saves state 
4. A runner that can handle pause/result 



## Building the Workflow

The workflow code uses ADK concepts like Sessions, Runners, and Events. 


### Key Tech Concepts

* Events: ADK creates events as the agent executes. Tool calls, model responses, function results - all become events.

* adk_request_confirmation_event: This is a special event - it signals "pause here"

   * Automatically created by ADK when your tool calls request_confirmation()
   * Contains the invocation_id
   * Your workflow must detect this event to know the agent paused
 
* invocation_id: Every call to `run_sync()` gets a unique `invocation_id` (like "abc123")

   * When a tool pauses, you save this ID
   * When resuming, pass the same ID so ADK knows which execution to continue
   * Without it, ADK would start a NEW execution instead of resuming the paused one
 
### Helper Functions to Process Events
These handles the event iteration logic.

`check_for_approval()` - detects if the agent paused


* Loops through all events and looks for the special adk_request_confirmation event
* Returns approval_id (identifies this specific request) and invocation_id (identifies which execution to resume)
* Returns None if no pause detected


                             

In [ ]:
def check_for_approval(events):
    """Check if events contain an approval request.

    Returns:
        dict with approval details or None
    """
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if (
                    part.function_call
                    and part.function_call.name == "adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }
    return None

`print_agent_response()` - Displays agent text

* Simple helper to extract and print text from events

In [ ]:
def print_agent_response(events):
    """Print agent's text responses from events."""
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

`create_approval_response()` - Formats the human decision

* Takes the approval info and boolean decision (True/False) from the human
* Creates a FunctionResponse that ADK understands
* Wraps it in a Content object to send back to the agent

In [ ]:
def create_approval_response(approval_info, approved):
    """Create approval response message."""
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={"confirmed": approved},
    )
    return types.Content(
        role="user", parts=[types.Part(function_response=confirmation_response)]
    )


In [ ]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    """Runs a shipping workflow with approval handling.

    Args:
        query: User's shipping request
        auto_approve: Whether to auto-approve large orders (simulates human decision)
    """

    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    # Generate unique session ID
    session_id = f"order_{uuid.uuid4().hex[:8]}"

    # Create session
    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events = []

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
    approval_info = check_for_approval(events)

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        # PATH A: Resume the agent by calling run_async() again with the approval decision
        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id=session_id,
            new_message=create_approval_response(
                approval_info, auto_approve
            ),  # Send human decision here
            invocation_id=approval_info[
                "invocation_id"
            ],  # Critical: same invocation_id tells ADK to RESUME
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    else:
        # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
        print_agent_response(events)

    print(f"{'='*60}\n")


print("✅ Workflow function ready")

### Code breakdown¶

* Step 1: Send initial request to the Agent

Call run_async() to start agent execution
Collect all events in a list for inspection

* Step 2: Detect Pause

Call check_for_approval(events) to look for the special event: adk_request_confirmation
Returns approval info (with invocation_id) if the special event is present; None if completed


* Step 3: Resume execution

PATH A:

If the approval info is present, at this point the Agent pauses for human input.
Once the Human input is available, call the agent again using run_async() and pass in the Human input.
Critical: Same invocation_id (tells ADK to RESUME, not restart)
Display agent's final response after resuming

PATH B:

If the approval info is not present, then approval is not needed and the agent completes execution.